In [2]:
import tensorflow as tf
import tqdm
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import zipfile
import numpy as np
from tqdm import tqdm_notebook
%matplotlib inline
import sys
sys.modules['Image'] = Image 
tf.__version__

'2.1.0'

In [ ]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

### Unziping the dataset

In [2]:
zip_dataset = 'C:/Users/anurag/Downloads/Compressed/cats_and_dogs_filtered.zip'

In [3]:
zip_object = zipfile.ZipFile(file=zip_dataset, mode="r")

In [4]:
zip_object.extractall('D:/DataScience/DataSets/')

In [5]:
zip_object.close()

### Setting up the dataset

In [3]:
dataset_path = 'D:/DataScience/DataSets/cats_and_dogs_filtered/'

In [4]:
train_dir = os.path.join(dataset_path,'train')
validation_dir = os.path.join(dataset_path,'validation')

### Building the Model

#### Loading the pre-trained model

In [5]:
Image_shape = (128,128,3)

In [6]:
base_model = tf.keras.applications.MobileNetV2(input_shape=Image_shape, include_top=False, weights="imagenet")

In [10]:
#base_model.summary()

#### Freezing the model

In [7]:
base_model.trainable = False

#### Adding custom head to the model

In [8]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

https://stackoverflow.com/questions/49295311/what-is-the-difference-between-flatten-and-globalaveragepooling2d-in-keras

In [9]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [10]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [11]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

#### Defining the Model

In [12]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [13]:
#model.summary()

#### Compiling the Model

In [14]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

#### Creating data generator 

In [15]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [16]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [17]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=128, class_mode="binary")

Found 1000 images belonging to 2 classes.


#### Training the model

In [18]:
model.fit(train_generator, epochs=2, validation_data=valid_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 16 steps, validate for 8 steps
Epoch 1/2
16/16 [==============================] - 79s 5s/step - loss: 0.7530 - accuracy: 0.5495 - val_loss: 0.5788 - val_accuracy: 0.7200
Epoch 2/2
16/16 [==============================] - 79s 5s/step - loss: 0.6776 - accuracy: 0.6075 - val_loss: 0.5136 - val_accuracy: 0.7750


In [19]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Instructions for updating:
Please use Model.evaluate, which supports generators.
  ...
    to  
  ['...']
Accuracy after transfer learning: 0.7749999761581421


### Unfreeze the top layers

In [20]:
base_model.trainable = True

In [21]:
fine_tune_at = 100

In [22]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

In [23]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [24]:
model.fit(train_generator,  
                    epochs=5, 
                    validation_data=valid_generator)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 16 steps, validate for 8 steps
Epoch 1/5
16/16 [==============================] - 97s 6s/step - loss: 0.2001 - accuracy: 0.9225 - val_loss: 0.0897 - val_accuracy: 0.9690
Epoch 2/5
16/16 [==============================] - 88s 5s/step - loss: 0.0223 - accuracy: 0.9990 - val_loss: 0.0964 - val_accuracy: 0.9670
Epoch 3/5
16/16 [==============================] - 95s 6s/step - loss: 0.0072 - accuracy: 1.0000 - val_loss: 0.1070 - val_accuracy: 0.9690
Epoch 4/5
16/16 [==============================] - 90s 6s/step - loss: 0.0025 - accuracy: 1.0000 - val_loss: 0.1265 - val_accuracy: 0.9640
Epoch 5/5
16/16 [==============================] - 93s 6s/step - loss: 9.2564e-04 - accuracy: 1.0000 - val_loss: 0.1449 - val_accuracy: 0.9630
